<a href="https://colab.research.google.com/github/JavaFXpert/ReActTextWorldLangChain/blob/main/TextWorld_React_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TextWorld React Agent
Notebook for creating and playing a [TextWorld](https://www.microsoft.com/en-us/research/project/textworld/) game with a LangChain ReAct agent and OpenAI GPT-3.

## Prerequisites
Install the TextWorld [README.md](https://github.com/microsoft/TextWorld#readme), [LangChain](https://github.com/hwchase17/langchain) and [OpenAI](https://openai.com/) libraries.

In [ ]:
!pip install textworld

In [ ]:
!pip install --pre -i https://test.pypi.org/simple  langchain==0.0.27

In [ ]:
!pip install openai

## Generate a TextWorld game
We'll use the [tw-make](https://textworld.readthedocs.io/en/stable/tw-make.html) script to define and generate a TextWorld game.

In [ ]:
import textworld
!tw-make custom --world-size 5 --quest-length 6 --nb-objects 10 --output tw_games/game.ulx -f -v --seed 456

In [ ]:
# Let the environment know what information we want as part of the game state.
infos = textworld.EnvInfos(
    feedback=True,    # Response from the game after typing a text command.
    description=True, # Text describing the room the player is currently in.
    inventory=True    # Text describing the player's inventory.
)

In [ ]:
# Get a reference to the game environment.
TW_ENV = textworld.start('tw_games/game.ulx', infos)

## Create a LangChain Tool to interact with the TextWorld game environment

In [ ]:
import sys
class HiddenPrints:
    """Context manager to hide prints."""

    def __enter__(self) -> None:
        """Open file to pipe stdout to."""
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, "w")

    def __exit__(self, *_) -> None:
        """Close file that stdout was piped to."""
        sys.stdout.close()
        sys.stdout = self._original_stdout

class TextWorldInteract:
    """Class to interact with a TextWorld game."""

    def __init__(self, textworld_env):
        """Initialize with the TextWorld game environment."""
        self.textworld_env = textworld_env

    def play(self, command) -> str:
        """Process a command and render the result."""
        if self.textworld_env is None:
            raise ValueError("TextWorld game environment not set")
        with HiddenPrints():
          game_state, reward, done = self.textworld_env.step(command)    
        return game_state["feedback"]       


In [ ]:
from langchain.agents import Tool
textworld_tool=TextWorldInteract(TW_ENV)
tools = [
    Tool(
        name = "Play",
        func = textworld_tool.play,
        description="useful for interacting with a TextWorld game environment"
    )
]

## Set up ReActTextWorld Agent and play the game

In [ ]:
# Supply your OpenAI API key
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain.agents import ReActTextWorldAgent
from langchain import OpenAI
llm = OpenAI(temperature=0)

In [ ]:
react = ReActTextWorldAgent.from_llm_and_tools(llm, tools, verbose=True)

In [ ]:
game_state = TW_ENV.reset()
init_text = game_state.objective + "\n\n" + game_state.description
react.run(init_text)



> Entering new ReActTextWorldAgent chain...
Hey, thanks for coming over to the TextWorld today, there is something I need you to do for me. First step, pick up the key from the rectangular locker. After you have got the key, make sure that the passageway in the cookery is unlocked. And then, open the passageway. Then, head east. With that done, retrieve the latchkey from the counter. Then, check that the chest in the attic is unlocked with the latchkey. And once you've done that, you win!

-= Cookery =-
You've just walked into a cookery.

You can see a rectangular locker. The rectangular locker contains a key, a rectangular keycard and a knife. You see an opened cabinet. Empty! What kind of nightmare TextWorld is this?

There is a closed passageway leading east. You need an unblocked exit? You should try going north.
Thought 1: I need to pick up the key from the rectangular locker
Action 1: Play[take key]
Observation 1: You take the key from the rectangular locker.


Thought 2: I nee

'yes'